## CLEANING

### Imports

In [1]:
import pandas as pd
import numpy as np

### Database loading

In [2]:
import sqlalchemy as db
DATABASE_URL = 'sqlite:///' + 'data.db'
engine = db.create_engine(DATABASE_URL, echo = True)
connection = engine.connect()

2020-05-29 00:17:55,097 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2020-05-29 00:17:55,098 INFO sqlalchemy.engine.base.Engine ()
2020-05-29 00:17:55,099 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2020-05-29 00:17:55,100 INFO sqlalchemy.engine.base.Engine ()


In [3]:
df = pd.read_sql('Data', con = engine)

2020-05-29 00:17:55,116 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("Data")
2020-05-29 00:17:55,120 INFO sqlalchemy.engine.base.Engine ()
2020-05-29 00:17:55,137 INFO sqlalchemy.engine.base.Engine SELECT name FROM sqlite_master WHERE type='table' ORDER BY name
2020-05-29 00:17:55,143 INFO sqlalchemy.engine.base.Engine ()
2020-05-29 00:17:55,151 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("Data")
2020-05-29 00:17:55,162 INFO sqlalchemy.engine.base.Engine ()
2020-05-29 00:17:55,176 INFO sqlalchemy.engine.base.Engine SELECT sql FROM  (SELECT * FROM sqlite_master UNION ALL   SELECT * FROM sqlite_temp_master) WHERE name = 'Data' AND type = 'table'
2020-05-29 00:17:55,182 INFO sqlalchemy.engine.base.Engine ()
2020-05-29 00:17:55,204 INFO sqlalchemy.engine.base.Engine PRAGMA main.foreign_key_list("Data")
2020-05-29 00:17:55,209 INFO sqlalchemy.engine.base.Engine ()
2020-05-29 00:17:55,213 INFO sqlalchemy.engine.base.Engine PRAGMA temp.foreign_key_list("Data")
2020-

### Data cleaning

In [5]:
df_c = df.copy()

In [6]:
df_c.columns

Index(['index', 'ID', 'NUMERO_AFFAIRE', 'DATE_DEC', 'NATURE', 'SIEGE_APPEL',
       'CONTENU', 'SCT'],
      dtype='object')

#### Index

In [7]:
df_c.drop('index', axis = 1, inplace = True)

#### NUMERO_AFFAIRE

In [8]:
df_c['NUMERO_AFFAIRE'][df_c['NUMERO_AFFAIRE'] == 'null'] = np.NaN

#### DATE_DEC

In [9]:
date_typo = ['0201-04-04', '0201-11-23', '0201-04-07', '0201-08-24', '0201-06-28', '0201-02-24']

In [10]:
for i in date_typo:
    c = i.split('-')
    c[0] = '2011'
    c = '-'.join(c)
    df_c['DATE_DEC'][df_c['DATE_DEC'] == i] = c

In [11]:
df_c['DATE_DEC'] = pd.to_datetime(df_c['DATE_DEC'], format = '%Y-%m-%d')

#### NATURE

In [12]:
df_c['NATURE'][df_c['NATURE'] == 'null'] = np.NaN

/Users/leoguillaume/.pyenv/versions/3.7.6/envs/vivadata/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


#### SIEGE_APPEL

In [13]:
df_c['SIEGE_APPEL'][df_c['SIEGE_APPEL'] == 'null'] = np.NaN

/Users/leoguillaume/.pyenv/versions/3.7.6/envs/vivadata/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


#### SCT

In [14]:
df_c['SCT'][(df_c['SCT'] == 'null') | 
            (df_c['SCT'] == 'null null') | 
            (df_c['SCT'] == 'a\n\n') |
            (df_c['SCT'] == '  null') |
            (df_c['SCT'] == '\n\n null') |
            (df_c['SCT'] == '\n\n\n') |
            (df_c['SCT'] == 'null null null null null null') |
            (df_c['SCT'] == 'null null null null') |
            (df_c['SCT'] == 'null null null') |
            (df_c['SCT'] == 'A SUPPRIMER') |
            (df_c['SCT'] == '/JDF\n null') |
            (df_c['SCT'] == 'A SUPPRIMER\n null A SUPPRIMER\n null') |
            (df_c['SCT'] == 'adoublon avec 2000/01644\n\n') |
            (df_c['SCT'] == 'adoublon avec 2002/00020\n\n') |
            (df_c['SCT'] == '/\n null')] = np.NaN

/Users/leoguillaume/.pyenv/versions/3.7.6/envs/vivadata/lib/python3.7/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app


In [15]:
df_c.to_pickle("data.pkl")